#Assigment 3: Social Network Analysis for U.S presidential elections
<br>42913 Social and Information Network Analysis - Autumn 2020

<br>Carlos Mario Carvajal Moreno. Student ID: 13144148
<br>Andres Felipe Lagos. Student ID: 13092248
<br>Ernest Ilustre.  Student ID: 12763239

# Sentiment Analysis of data collected

## TextBlob and other libraries

The following libraries were imported into the python file in order to wrangle the data to the desire format, with the desire observations and 

In [17]:
pip install twython

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os
from textblob import TextBlob
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import zipfile
pd.options.mode.chained_assignment = "warn"

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Gathering all the tweets collected over a 5 day period.

In [0]:
#Unziping the downloaded file and placing the images in the tmp folder
local_zip = '/content/drive/My Drive/tweets.txt.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [0]:
# String of path to file: tweets_data_path
tweets_data_path = ["/content/tweets.txt"]

# Initialize empty list to store tweets: tweets_data
tweets_data = []

# Open connection to files with a for loop
for data in tweets_data_path:
  tweets_file = open(data, "r")
  # Read in tweets and store in list: tweets_data
  for line in tweets_file:
      tweet = json.loads(line)
      tweets_data.append(tweet)
# Close connection to file
tweets_file.close()

# Creating the Pandas dataframe

In [0]:
# Creating an empty dictionary to append the tweets
dict_ = {'Tweet_id': [], 'User_id':[], 'user': [], 'date': [], 'text': [],
         "followers_count": [], 'Place':[], 'Language': []
         }


# For loop to append data into the dictionary        
for status in tweets_data:
    dict_['Tweet_id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['User_id'].append(status['user']['id_str'])
    dict_['date'].append(status['created_at'])
    if 'extended_tweet' in status.keys():
      dict_['text'].append(status['extended_tweet']['full_text'])
    else:
      dict_['text'].append(status['text'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['Place'].append(status['user']['location'])
    dict_['Language'].append(status['lang'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.head()

,Tweet_id,User_id,user,date,text,followers_count,Place,Language
0,1271202831198851074,1220729068858961920,IndependentThat,Thu Jun 11 22:09:07 +0000 2020,@IAMChrisEdward Trump definitely knows how to ...,14,"Seattle, WA",en
1,1271202830704132097,22781602,s1eddog,Thu Jun 11 22:09:07 +0000 2020,Here’s a screenshot from the Tulsa rally signu...,16,"Madison, WI",en
2,1271202831752560640,21535453,jssacramento,Thu Jun 11 22:09:07 +0000 2020,@Tee2019K @WeStand4theFlag @classybass3 @Whale...,3890,None,tl
3,1271202833006821381,1261094227464904704,painterjoe24,Thu Jun 11 22:09:07 +0000 2020,@oliviawilde Ivanka for president... If this w...,3,None,en
4,1271202833120014343,1478689148,RickPetree,Thu Jun 11 22:09:07 +0000 2020,@cincity631 @iosolofede @NastyOldWomyn @delacr...,17085,New York City,en


In [0]:
#Checking the number of tweets. 
len(df)

68426

# Data cleaning Pipeline

Defining a reading function to identify specific words in fields in the tweets data

In [0]:
#Function that performs a read of a field and finds a word. It returns a Boolean
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)

    if match:
        return True
    return False

#Defining a function that matches the exact word.
def find_word(text, search):
   result = re.findall('\\b'+search+'\\b', text, flags=re.IGNORECASE)
   if len(result)>0:
      return True
   else:
      return False

Performing cleaning of data

In [0]:
#Removing duplicates (tweets with the same id)
df = df.drop_duplicates(subset = "Tweet_id", ignore_index = True)

#Removing tweets that are not in english:
df = df[df['Language'] == 'en']

#Localising tweets that contain the word Trump
#Initializing columns with 0 values
df['Trump'] = 0
df['Biden'] = 0
df['Both']= 0

#Creating column Trump if trump is mentioned in tweet
for (index,row) in df.iterrows():
  if  str(word_in_text('Trump', df['text'][index])) == 'True':
    df['Trump'][index] = 'Trump'
  else:
    df['Trump'][index] = False

#Creating column Biden if Biden is mentioned in tweet
for (index,row) in df.iterrows():
  if  str(word_in_text('Biden', df['text'][index])) == 'True':
    df['Biden'][index] = 'Biden'
  else:
    df['Biden'][index] = False


#Creating column Trump and Biden if Biden is mentioned in tweet
for (index,row) in df.iterrows():
  if  str(word_in_text('Biden', df['text'][index])) == 'True':
    if  str(word_in_text('Trump', df['text'][index])) == 'True':
      df['Both'][index] = 'Trump and Biden'
    else:
      df['Both'][index] = False

#Digging deeper to find the real mention in the tweet
for (index,row) in df.iterrows():
  if df['Both'][index] == 'Trump and Biden':
    if  str(find_word(str(row.loc['text']), 'Biden')) == 'True' and str(find_word(str(row.loc['text']), 'Trump')) == 'True':
      df['Both'][index] = 'Trump and Biden'
    elif  str(find_word(str(row.loc['text']), 'Biden')) == 'False' and str(find_word(str(row.loc['text']), 'Trump')) == 'True':
      df['Both'][index] = 'Trump'
    elif  str(find_word(str(row.loc['text']), 'Biden')) == 'True' and str(find_word(str(row.loc['text']), 'Trump')) == 'False':
      df['Both'][index] = 'Biden'

#Creating an integrated column
df['Mention'] = 0

for (index,row) in df.iterrows():
  if  df['Both'][index] == 'Trump and Biden':
    df['Mention'][index] = 'Trump and Biden'
  elif df['Both'][index] == 'Trump':
    df['Mention'][index] = 'Trump'
  elif df['Both'][index] == 'Biden':
    df['Mention'][index] = 'Biden'    
  elif df['Trump'][index] == 'Trump':
    df['Mention'][index] = 'Trump'
  elif df['Biden'][index] == 'Biden':
    df['Mention'][index] = 'Biden'
  else:
    df['Mention'][index] = 'No mention'

#Droping the created columns leaving just mention of trump or biden.
df = df.drop(['Trump','Biden','Both'], axis = 1)
df = df[df['Mention'] != 'No mention']


#Assigning United States to places where location is unknown, with the assumption of
#People talking about Trump in English right now has a high likelihood of being an american.
#Creating column location with cities and states dataset

df["Place"] = df["Place"].fillna('United States')

#Filtering out places not in the U.S:
#Bringing a list of all states and cities in the US, in a CSV file
usa = pd.read_csv('us.csv', index_col= False)

#storing locations in a list format
usa_locations = list(usa['Location'])

df['Location'] = 0
#Creating column Location to append locations similarity to dataset
for (index,row) in df.iterrows():
  for location in usa_locations:
    if  str(find_word(str(row.loc['Place']), location)) == 'True':
      df['Location'][index] = location
      break
    else:
      df['Location'][index] = "Na"

#Dropping places that not correspond to a state in the US
df = df[df['Location'] !='Na']

#Converting the locations to States
states = pd.DataFrame(pd.read_csv('states.csv', index_col= False))
states =  dict(zip(states.Location, states.State))

df['State'] = 0

for (index,row) in df.iterrows():
  for k,v in states.items():
    if  str(find_word(row.loc['Location'], k)) == 'True':
      df['State'][index] = v
      break
    else:
      df['State'][index] = "Na"

#Droping columns that are not needed.
df = df.drop(['Location'], axis =1)
df = df.reset_index()
df = df.drop(['index'], axis = 1)

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packag

,Tweet_id,User_id,user,date,text,followers_count,Place,Language,Mention,State
0,1271202831198851074,1220729068858961920,IndependentThat,Thu Jun 11 22:09:07 +0000 2020,@IAMChrisEdward Trump definitely knows how to ...,14,"Seattle, WA",en,Trump,Washington
1,1271202830704132097,22781602,s1eddog,Thu Jun 11 22:09:07 +0000 2020,Here’s a screenshot from the Tulsa rally signu...,16,"Madison, WI",en,Trump,Wisconsin
2,1271202833006821381,1261094227464904704,painterjoe24,Thu Jun 11 22:09:07 +0000 2020,@oliviawilde Ivanka for president... If this w...,3,United States,en,Trump,United States
3,1271202833120014343,1478689148,RickPetree,Thu Jun 11 22:09:07 +0000 2020,@cincity631 @iosolofede @NastyOldWomyn @delacr...,17085,New York City,en,Trump,New York
4,1271202833724030982,2862562517,johnkrol60,Thu Jun 11 22:09:08 +0000 2020,@CNN well nobody cared about that when they we...,5,United States,en,Trump,United States


In [0]:
#Checking the number of tweets. 
len(df)

47549

#Sentiment Analysis

## TextBlob

In [0]:
df['polarity'] = 0
df['subjectivity'] = 0
df["polarity"] = pd.to_numeric(df["polarity"], downcast="float")
df["subjectivity"] = pd.to_numeric(df["subjectivity"], downcast="float")

for index, row in df.iterrows():
  polarity, subjectivity = TextBlob(df['text'][index]).sentiment
  df['polarity'][index] = polarity
  df['subjectivity'][index] = subjectivity

df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Tweet_id,User_id,user,date,text,followers_count,Place,Language,Mention,State,polarity,subjectivity
0,1271202831198851074,1220729068858961920,IndependentThat,Thu Jun 11 22:09:07 +0000 2020,@IAMChrisEdward Trump definitely knows how to ...,14,"Seattle, WA",en,Trump,Washington,-0.050000,0.531250
1,1271202830704132097,22781602,s1eddog,Thu Jun 11 22:09:07 +0000 2020,Here’s a screenshot from the Tulsa rally signu...,16,"Madison, WI",en,Trump,Wisconsin,0.600000,1.000000
2,1271202833006821381,1261094227464904704,painterjoe24,Thu Jun 11 22:09:07 +0000 2020,@oliviawilde Ivanka for president... If this w...,3,United States,en,Trump,United States,-0.218750,0.575000
3,1271202833120014343,1478689148,RickPetree,Thu Jun 11 22:09:07 +0000 2020,@cincity631 @iosolofede @NastyOldWomyn @delacr...,17085,New York City,en,Trump,New York,0.008333,0.375641
4,1271202833724030982,2862562517,johnkrol60,Thu Jun 11 22:09:08 +0000 2020,@CNN well nobody cared about that when they we...,5,United States,en,Trump,United States,0.000000,0.000000


## NLTK Vader Sentiment

In [0]:
#Initializing the NLTK sentiment analysis
sid = SentimentIntensityAnalyzer()

df['Compound'] = 0.0

for index, row in df.iterrows():
  compound = sid.polarity_scores(df['text'][index])['compound']
  df['Compound'][index] = compound


#Averaging the scores of both NLP analyses
df['Sentiment Score'] = (df['Compound']+df['polarity'])/2


df['Sentiment'] = 0

for index, row in df.iterrows():
  # decide sentiment as positive, negative and neutral 
  if df['Sentiment Score'][index] >= 0.05 :
    df['Sentiment'][index] = 'Positive' 
  elif df['Sentiment Score'][index] <= - 0.05 :
    df['Sentiment'][index] = 'Negative' 
  else :
    df['Sentiment'][index] = 'Neutral'


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2

In [0]:
df

,Tweet_id,User_id,user,date,text,followers_count,Place,Language,Mention,State,polarity,subjectivity,Compound,Sentiment Score,Sentiment
0,1271202831198851074,1220729068858961920,IndependentThat,Thu Jun 11 22:09:07 +0000 2020,@IAMChrisEdward Trump definitely knows how to ...,14,"Seattle, WA",en,Trump,Washington,-0.050000,0.531250,-0.6360,-0.343000,Negative
1,1271202830704132097,22781602,s1eddog,Thu Jun 11 22:09:07 +0000 2020,Here’s a screenshot from the Tulsa rally signu...,16,"Madison, WI",en,Trump,Wisconsin,0.600000,1.000000,-0.2732,0.163400,Positive
2,1271202833006821381,1261094227464904704,painterjoe24,Thu Jun 11 22:09:07 +0000 2020,@oliviawilde Ivanka for president... If this w...,3,United States,en,Trump,United States,-0.218750,0.575000,-0.6597,-0.439225,Negative
3,1271202833120014343,1478689148,RickPetree,Thu Jun 11 22:09:07 +0000 2020,@cincity631 @iosolofede @NastyOldWomyn @delacr...,17085,New York City,en,Trump,New York,0.008333,0.375641,-0.5719,-0.281783,Negative
4,1271202833724030982,2862562517,johnkrol60,Thu Jun 11 22:09:08 +0000 2020,@CNN well nobody cared about that when they we...,5,United States,en,Trump,United States,0.000000,0.000000,-0.8399,-0.419950,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47544,1271268092027404289,42048593,nanatoyou,Fri Jun 12 02:28:26 +0000 2020,@HoarseWisperer @Alt_FedEmployee Trump needs m...,7093,United States,en,Trump,United States,-0.062500,0.525000,0.0000,-0.031250,Neutral
47545,1271268092165980162,1522327446,lee_myrna,Fri Jun 12 02:28:26 +0000 2020,@ewarren @HolderJeannehom It was a Trump/Repub...,625,"Illinois, USA",en,Trump,Illinois,0.250000,0.750000,0.4404,0.345200,Positive
47546,1271268093247909889,1017509607013474305,TheyCallMeTomO1,Fri Jun 12 02:28:27 +0000 2020,No matter how dhitty the Dems treat President ...,33136,"Sacramento, CA",en,Trump,California,0.000000,0.000000,0.2244,0.112200,Positive
47547,1271268093424070657,746377806754394116,BruceDuck2,Fri Jun 12 02:28:27 +0000 2020,@shayleesbibi @ThomasM64252649 @Thom_Sgt @mike...,1445,"Oregon, USA",en,Trump,Oregon,0.600000,0.300000,0.6697,0.634850,Positive


In [0]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
from google.colab import files
df.to_csv('tweeter_dataset2.csv')

In [0]:
files.download('tweeter_dataset2.csv')

# Analysis of Influencers



Loading the compiled dataset

In [3]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os
from textblob import TextBlob
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import zipfile
pd.options.mode.chained_assignment = "warn"

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [0]:
#Unziping the downloaded file and placing the images in the tmp folder
local_zip = '//content/drive/My Drive/tweeter_dataset.csv.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
df = pd.DataFrame(pd.read_csv("/content/tweeter_dataset.csv"))
df

,Tweet_id,User_id,date,text,followers_count,Mention,State,polarity,subjectivity,Compound,Sentiment Score,Sentiment
0,1.270000e+18,9.630000e+17,Sun Jun 07 08:51:07 +0000 2020,@StewieGilchrist The problem is they turn out ...,196,Trump,United States,-0.400000,0.500000,-0.7197,-0.559850,Negative
1,1.270000e+18,3.840433e+07,Sun Jun 07 08:51:07 +0000 2020,"I just posted ""Media And Democrats ""Political ...",1615,Trump,United States,0.000000,0.550000,-0.2577,-0.128850,Negative
2,1.270000e+18,1.709346e+09,Sun Jun 07 08:51:10 +0000 2020,Stand Together for the USA regardless of Party...,110,Trump,Colorado,0.296875,0.637500,0.8306,0.563738,Positive
3,1.270000e+18,2.470771e+09,Sun Jun 07 08:51:10 +0000 2020,"@IngrahamAngle @AalstV Trump has a Bible, I'm ...",18,Trump,United States,0.037500,0.734722,0.0516,0.044550,Neutral
4,1.270000e+18,1.170000e+18,Sun Jun 07 08:51:11 +0000 2020,@edisn @igobyMdot @KyaeGunn @HillaryClinton I'...,121,Trump,United States,-0.075000,0.516667,0.2168,0.070900,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
432810,1.270000e+18,4.204859e+07,Fri Jun 12 02:28:26 +0000 2020,@HoarseWisperer @Alt_FedEmployee Trump needs m...,7093,Trump,United States,-0.062500,0.525000,0.0000,-0.031250,Neutral
432811,1.270000e+18,1.522327e+09,Fri Jun 12 02:28:26 +0000 2020,@ewarren @HolderJeannehom It was a Trump/Repub...,625,Trump,Illinois,0.250000,0.750000,0.4404,0.345200,Positive
432812,1.270000e+18,1.020000e+18,Fri Jun 12 02:28:27 +0000 2020,No matter how dhitty the Dems treat President ...,33136,Trump,California,0.000000,0.000000,0.2244,0.112200,Positive
432813,1.270000e+18,7.460000e+17,Fri Jun 12 02:28:27 +0000 2020,@shayleesbibi @ThomasM64252649 @Thom_Sgt @mike...,1445,Trump,Oregon,0.600000,0.300000,0.6697,0.634850,Positive


In [6]:
df1 = df.drop_duplicates(['User_id'], keep = 'last', ignore_index=True)
df1

,Tweet_id,User_id,date,text,followers_count,Mention,State,polarity,subjectivity,Compound,Sentiment Score,Sentiment
0,1.270000e+18,3.840433e+07,Sun Jun 07 08:51:07 +0000 2020,"I just posted ""Media And Democrats ""Political ...",1615,Trump,United States,0.000000,0.550000,-0.2577,-0.128850,Negative
1,1.270000e+18,7.386861e+07,Sun Jun 07 08:51:20 +0000 2020,#GeorgeLloyd #coronavirus #JK #jk_rowling #Don...,270,Trump,United States,0.133333,0.533333,-0.3034,-0.085033,Negative
2,1.270000e+18,3.485457e+09,Sun Jun 07 08:51:25 +0000 2020,@brandondarby You have been played by the demo...,202,Trump,Georgia,0.466667,0.700000,0.8271,0.646883,Positive
3,1.270000e+18,2.595886e+07,Sun Jun 07 08:51:26 +0000 2020,@WaltBlackmanAZ What is your malfunction?! I‚Ä...,4,Trump,United States,-0.500000,0.950000,-0.8070,-0.653500,Negative
4,1.270000e+18,5.035796e+08,Sun Jun 07 08:51:31 +0000 2020,@itsGraceNoriega When you stop calling my comm...,247,Trump,United States,0.378788,0.766667,-0.0516,0.163594,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...
110441,1.270000e+18,4.204859e+07,Fri Jun 12 02:28:26 +0000 2020,@HoarseWisperer @Alt_FedEmployee Trump needs m...,7093,Trump,United States,-0.062500,0.525000,0.0000,-0.031250,Neutral
110442,1.270000e+18,1.522327e+09,Fri Jun 12 02:28:26 +0000 2020,@ewarren @HolderJeannehom It was a Trump/Repub...,625,Trump,Illinois,0.250000,0.750000,0.4404,0.345200,Positive
110443,1.270000e+18,1.020000e+18,Fri Jun 12 02:28:27 +0000 2020,No matter how dhitty the Dems treat President ...,33136,Trump,California,0.000000,0.000000,0.2244,0.112200,Positive
110444,1.270000e+18,7.460000e+17,Fri Jun 12 02:28:27 +0000 2020,@shayleesbibi @ThomasM64252649 @Thom_Sgt @mike...,1445,Trump,Oregon,0.600000,0.300000,0.6697,0.634850,Positive


In [8]:
x = df1[df1['Mention'] == 'Trump']['Sentiment'].value_counts()
print("Trump's sentiment votes")
x

Trump's sentiment votes


Negative    44994
Positive    36831
Neutral      9098
Name: Sentiment, dtype: int64

In [9]:
y = df1[df1['Mention'] == 'Biden']['Sentiment'].value_counts()
print("Biden's sentiment votes")
y

Biden's sentiment votes


Positive    6163
Negative    5790
Neutral     1445
Name: Sentiment, dtype: int64

In [13]:
t = np.round((36831+5790)/(x.sum()+y.sum())*100,2)
b = np.round((6163+44994)/(x.sum()+y.sum())*100,2)

print("National Results by accounting only unique twitter accounts as voters")
print("Trump: {}%".format(t))
print("Biden: {}%".format(b))


National Results by accounting only unique twitter accounts as voters
Trump: 40.86%
Biden: 49.04%


In [14]:
df1['Trump Voters'] = 0
df1['Biden Voters'] = 0
df1['Neutral Voters'] = 0

proportion = 0.0005
influence = 5000

for index, row in df1.iterrows():
  if df1['Mention'][index] == 'Trump' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Positive':
    df1['Trump Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Trump' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Positive':
    df1['Trump Voters'][index] = 1
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Negative':
    df1['Trump Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Negative':
    df1['Trump Voters'][index] = 1
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Positive':
    df1['Biden Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Positive':
    df1['Biden Voters'][index] = 1
  elif df1['Mention'][index] == 'Trump' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Negative':
    df1['Biden Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Trump' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Negative':
    df1['Biden Voters'][index] = 1
  elif df1['Mention'][index] == 'Trump' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = 1
  elif df1['Mention'][index] == 'Trump' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = 1
  elif df1['Mention'][index] == 'Biden' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = df1['followers_count'][index]*proportion
  elif df1['Mention'][index] == 'Trump & Biden' and df1['followers_count'][index] < influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = 1
  elif df1['Mention'][index] == 'Trump & Biden' and df1['followers_count'][index] >= influence and df1['Sentiment'][index] == 'Neutral':
    df1['Neutral Voters'][index] = df1['followers_count'][index]*proportion
  else:
    df1['Neutral Voters'][index] = 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [0]:
Florida = df1[df1['State'] == 'Florida']
Ohio = df1[df1['State'] == 'Ohio']
NC = df1[df1['State'] == 'North Carolina']

In [16]:
df1['Trump Voters'].sum()

218045

In [17]:
df1['Biden Voters'].sum()

329271

In [18]:
trump = df1['Trump Voters'].sum()/(df1['Trump Voters'].sum()+df1['Biden Voters'].sum()+df1['Neutral Voters'].sum())*100
biden = df1['Biden Voters'].sum()/(df1['Trump Voters'].sum()+df1['Biden Voters'].sum()+df1['Neutral Voters'].sum())*100
print("The prediction for the 2020 election by influencer's analysis is:")
print('Biden: {}% \nTrump: {}%'.format(biden,trump))

The prediction for the 2020 election by influencer's analysis is:
Biden: 53.983808349618975% 
Trump: 35.74836378421626%


Florida

In [19]:
trump = Florida['Trump Voters'].sum()/(Florida['Trump Voters'].sum()+Florida['Biden Voters'].sum()+Florida['Neutral Voters'].sum())*100
biden = Florida['Biden Voters'].sum()/(Florida['Trump Voters'].sum()+Florida['Biden Voters'].sum()+Florida['Neutral Voters'].sum())*100

print('Biden: {}% \nTrump: {}%'.format(biden,trump))

Biden: 42.857142857142854% 
Trump: 41.50827713059473%


Ohio

In [217]:
trump = Ohio['Trump Voters'].sum()/(Ohio['Trump Voters'].sum()+Ohio['Biden Voters'].sum()+Ohio['Neutral Voters'].sum())*100
biden = Ohio['Biden Voters'].sum()/(Ohio['Trump Voters'].sum()+Ohio['Biden Voters'].sum()+Ohio['Neutral Voters'].sum())*100

print('Biden: {}% \nTrump: {}%'.format(biden,trump))

Biden: 38.29588014981273% 
Trump: 51.9350811485643%


North Carolina

In [218]:
trump = NC['Trump Voters'].sum()/(NC['Trump Voters'].sum()+NC['Biden Voters'].sum()+NC['Neutral Voters'].sum())*100
biden = NC['Biden Voters'].sum()/(NC['Trump Voters'].sum()+NC['Biden Voters'].sum()+NC['Neutral Voters'].sum())*100

print('Biden: {}% \nTrump: {}%'.format(biden,trump))

Biden: 42.41130968258202% 
Trump: 44.94531875166711%


The visual analysis is performed on PowerBI.

In [13]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
from google.colab import files
df1.to_csv('final_twitter_set_influencer.csv')